<a href="https://colab.research.google.com/github/AsifShaafi/Deep-Learning-Course-Examples/blob/main/RNN_Examples/Shaafi_CharacterGeneratorRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Natural Language Processing (Character Generation) with Recurrent Neural Network


We will generate charater using a RNN.  We will work with a dataset of Shakespeare's writing.  Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

- The model is character-based.
- The model is trained on small batches of text (100 characters each), and is able to generate a longer sequence of text with coherent structure.

*This guide is based on the following: https://www.tensorflow.org/tutorials/text/text_generation*


In [1]:
# %tensorflow_version 2.x
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

###Download the Shakespeare Dataset

Here, we use an extract from a Shadespheare play for training.  We can use our own text paragraph data and use it for network training.

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

# If you load your own data file, use the following lines:
# from google.colab import files
# path_to_file = list(files.upload().keys())[0]

1115394/1115394 [==============================] - 0s 0us/step


###Read Contents of File
Let's look at the contents of the file.

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


###Encode
Encode each unique character as a different integer.

In [6]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [7]:
# Chect how a part of the text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


Make a function that can convert the numeric values back to text.


In [8]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


###Create Training Examples

Given a character, or a sequence of characters, what is the most probable next character? This is the task we are training the model to perform.

Our task is to feed the model a sequence and have it return the next character. This means we need to split our text data from above into many shorter sequences that we can pass to the model as training examples.

The training examples we prepapre will use a *seq_length* sequence as input and a *seq_length* sequence as the output where that sequence is the original sequence shifted one letter to the right. For example:

```input: Hell | output: ello```

In [9]:
# Create a stream of characters from our text data
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

# Use the batch method to turn this stream of characters into batches of desired length
seq_length = 100  # length of sequence for a training example
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
# Use these sequences of length 101 and split them into input and output.
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [10]:
for x, y in dataset.take(3):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


EXAMPLE

INPUT
now Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us k

OUTPUT
ow Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us ki


Make training batches.

In [11]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

###Build the Model
We use an embedding layer, a LSTM layer, and one dense layer that contains a node for each unique character in our training data.  The dense layer will give us a probability distribution over all nodes.

In [12]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    # tf.keras.layers.LSTM(rnn_units,
    #                     return_sequences=True,
    #                     stateful=True,
    #                     recurrent_initializer='glorot_uniform'),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 gru (GRU)                   (64, None, 1024)          3938304   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 4021569 (15.34 MB)
Trainable params: 4021569 (15.34 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


###Create a Loss Function
Our model will output a (64, sequence_length, 65) shaped tensor that represents the probability distribution of each character at each timestep for every sequence in the batch.

We will create the loss function.

Before we do that, let's have a look at a sample input and the output from the untrained model.

In [13]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [14]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 7.16557913e-03  4.55094455e-03 -9.02098976e-03 ... -1.58771724e-02
    2.41876114e-03 -5.35585172e-03]
  [-4.48164111e-03  5.12673380e-03 -6.36939472e-03 ...  2.08509853e-03
    1.70038873e-03 -1.41482810e-02]
  [-3.86452163e-03  6.44045044e-03 -3.04444460e-03 ...  1.36579201e-03
    9.56401881e-03 -8.32972676e-03]
  ...
  [-1.14303082e-02  2.08841963e-03 -7.45146628e-03 ...  1.29658934e-02
    6.76052133e-03 -1.92925408e-02]
  [ 7.03531411e-03  1.45608385e-03  2.18321709e-03 ...  5.43441204e-03
    6.39632065e-03 -4.17192699e-03]
  [ 2.56577530e-03 -4.08131583e-03  9.24427062e-03 ...  1.84575235e-03
    4.06372081e-03 -1.01777073e-02]]

 [[-1.29635893e-02 -5.33036375e-03 -1.73923606e-03 ... -1.93335721e-03
   -1.37662264e-02  8.60194676e-03]
  [ 9.62834526e-03  4.90402570e-03 -4.33432916e-03 ...  1.70066580e-03
    1.56126102e-04 -2.51942733e-03]
  [-2.38981820e-03  4.07911744e-03 -3.09764291e-03 ...  1.07163964e-02
   -1.15511101e-03 -1.58080384e-02]
  ...
  [ 9.209

In [15]:
# Let's examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[ 0.00716558  0.00455094 -0.00902099 ... -0.01587717  0.00241876
  -0.00535585]
 [-0.00448164  0.00512673 -0.00636939 ...  0.0020851   0.00170039
  -0.01414828]
 [-0.00386452  0.00644045 -0.00304444 ...  0.00136579  0.00956402
  -0.00832973]
 ...
 [-0.01143031  0.00208842 -0.00745147 ...  0.01296589  0.00676052
  -0.01929254]
 [ 0.00703531  0.00145608  0.00218322 ...  0.00543441  0.00639632
  -0.00417193]
 [ 0.00256578 -0.00408132  0.00924427 ...  0.00184575  0.00406372
  -0.01017771]], shape=(100, 65), dtype=float32)


In [16]:
# and finally we look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[ 7.16557913e-03  4.55094455e-03 -9.02098976e-03 -4.99140657e-03
 -5.74266911e-03  9.09409858e-03 -2.47118995e-03  4.10062214e-03
 -3.96627188e-03 -6.53983140e-03  2.42277398e-03  2.73624039e-03
 -6.35110866e-03  2.21808907e-04  1.52443321e-02 -4.39757202e-03
 -9.97514697e-04  5.66006638e-04 -1.01281162e-02  1.68750761e-03
  6.25314517e-03  2.38169637e-03  3.66933132e-03  3.11286212e-03
 -1.86005677e-03 -4.67114442e-05  9.08682682e-03 -1.98560441e-03
  4.31544520e-03 -8.19254108e-03  3.10907280e-03  6.88211713e-03
  2.00593169e-03 -3.80309997e-03 -4.52737231e-03 -1.59227184e-03
  3.57302488e-03  2.09645703e-02 -8.61120410e-04 -6.76857540e-03
 -1.16781853e-02  1.77943101e-03  4.44963621e-03 -5.81254251e-03
 -1.05820149e-02  4.97275125e-03 -5.50183374e-03  1.67343635e-02
  9.59283486e-03 -1.14300977e-02 -1.15597410e-04 -4.60100081e-03
  2.68617459e-03 -1.09541975e-02  1.36335252e-03 -1.43713655e-03
  3.98525735e-03 -2.61727581e-03  1.73901010e-03  4.83763451e-03
  1.7813891

In [17]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"yw;IdF$eTjt!vJDb:PYhbxX'oipLHtH ,pOHj;sq,UxEe:-uBcZODECytASwTPxaIqyTzgX\nfC&t$Nr'kc&SoTtzYfadU?x3BT i"

So, we need to create a loss function that can compare that output to the expected output and give us some numeric value representing how close the two were.

In [18]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

###Compile the Model
We can think of our problem as a classification problem where the model predicts the probabillity of each unique letter coming next.


In [19]:
model.compile(optimizer='adam', loss=loss)

###Create Checkpoints
Now we are going to setup and configure our model to save checkpoinst as it trains. This will allow us to load our model from a checkpoint and continue training it.

In [20]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

###Train the Model
**If this is taking a while go to Runtime > Change Runtime Type and choose "GPU" under hardware accelerator.**

In [ ]:
history = model.fit(data, epochs=30, callbacks=[checkpoint_callback])

Epoch 1/30
172/172 [==============================] - 20s 65ms/step - loss: 2.7035
Epoch 2/30
 25/172 [===>..........................] - ETA: 8s - loss: 2.1270

###Load the Model
We'll rebuild the model from a checkpoint using a batch_size of 1 so that we can feed one piece of text to the model and have it make a prediction.

In [ ]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

Once the model finishes training, we can find the **lastest checkpoint** that stores the models weights using the following line.

In [ ]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

We can load **any checkpoint** we want by specifying the exact file to load.

In [ ]:
# checkpoint_num = 10
# model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
# model.build(tf.TensorShape([1, None]))

###Generate Text
Generate some text using any starting string we choose.

Generating text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.  Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.

![alt text](https://www.tensorflow.org/text/tutorials/images/text_generation_sampling.png)


In [ ]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension

      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: Baylor my love, the river is green.
Baylor my love, the river is green.

DUKE VINCENTIO:
I know not what to say.

ESCALUS:
So, sir: alass!

CLAUDIO:
But how like a deep wrong? God forgive;
The commonwealtheaven I have forgot; with
And bosspless'd by his majesty hands as fan you
With woman's lightness of the tongue of Montague.
Come, fellow, yet I come to me,
To horse! u great alling no better throne,
That you did mock my own, and some comfort from reverse come:
In your cut against yourself?
It is the rascal from him understanding; and,
great mp to taste prove in hazard!
O, he is even in force, justice; he had comest thou hast
Thus your untimely death upon asurpherity.

First Servant:
Whrelf, yet since I seet me, if I go.

ARCHBISH:
He will be known thy men and hangs unto Hich,
We was not more to bear against my scouls

ISABELLA:
Care not thy dear lord Oxford 


##Sources

1. Chollet François. Deep Learning with Python. Manning Publications Co., 2018.
2. “Text Generation with an RNN &nbsp;: &nbsp; TensorFlow Core.” TensorFlow, www.tensorflow.org/tutorials/text/text_generation.
3. “Understanding LSTM Networks.” Understanding LSTM Networks -- Colah's Blog, https://colah.github.io/posts/2015-08-Understanding-LSTMs/.